In [2]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

#Defining various data and distacne matrix


I=['1','2','3','4','5','6','7','8','9'] # Demand ID
J=['A','B','C','D','E','F']   #EMS ID
A=[10,5,6,2,5,1,3,5,6]  # the amount of demand at each locaton
R=[0,1,1,1,0,0,0,0,0]  # whether the demand location is located in rural areas： 1-Yes, 0-No

M= 30 # total urban demand
N= 13 # total rural demand
tot_pop=M+N # total demand in the study area
q=2   #total number of EMS stations to be built 
p=1 # the number fo EMS stations should open

#########################################################################################
#O-D distance matrix 
#J         A B  C  D  E  F
D=       [[0,10,12,10,19,10],#1 urban
          [12,10,9,10,10,26],#2 rural 
          [12,13,10,8,17,10],#3 rural 
          [10,10,18,10,13,7],#4 rural 
          [19,17,7,13,10,19],#5 urban 
          [17,26,15,14,9,10],#6 urban
          [19,10,16,5,12,15],#7 urban
          [10,13,16,10,7,14],#8 urban
          [12,15,6,18,10,13]]#9 urban 

# coverage=9 if location is covered -1, not covered -0

C=       [[1,0,0,0,0,0],#1-
          [0,0,1,0,0,0],#2 
          [0,0,0,1,0,0],#3 -
          [0,0,0,0,0,1],#4-
          [0,0,1,0,0,0],#5 
          [0,0,0,0,1,0],#6 -
          [0,0,0,1,0,0],#7-
          [0,0,0,0,1,0],#8-
          [0,0,1,0,0,0]]#9

#A=[10,5,6,2,5,1,3,5,6]
# if location is covered -1, not covered 0, 
UC=      [[0,1,1,1,1,1],#1
          [1,1,0,1,1,1],#2 
          [1,1,1,0,1,1],#3
          [1,1,1,1,1,0],#4
          [1,1,0,1,1,1],#5
          [1,1,1,1,0,1],#6
          [1,1,1,0,1,1],#7
          [1,1,1,1,0,1],#8
          [1,1,0,1,1,1]]#9

print("Finish")

Finish


In [12]:
#  Model 1 - maximize the amount of covered demand. 
#we aim to build two EMS station, that is q=2

import gurobipy as gp
from gurobipy import GRB
import numpy as np
m=gp.Model("Model1 ")

#define decision variables  
X = m.addVars(range(1,len(J)+1),vtype=GRB.BINARY,name="Xj") # covergae decision
Y = m.addVars(range(1,len(I)+1),range(1,len(J)+1),vtype=GRB.INTEGER,lb=0,name="Yij") #Allocation variables

# Define constraints:
#SumXj +Sum(uc)Yij >=1
m.addConstrs(gp.quicksum(X[j]*C[i-1][j-1] for j in range(1,len(J)+1))+
             gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1, len(J)+1))>=1 for i in range(1, len(I)+1))

#Sum Xj=q
m.addConstr(X.sum()==q)
m.addConstr(gp.quicksum(X[j] for j in range(2,3))>=p)

# X[j]-Y[i,j]>=0 for i in I, j in UC
m.addConstrs(X[j]*UC[i-1][j-1]-Y[i,j]*UC[i-1][j-1]>=0 for i in range(1,len(I)+1) for j in range(1,len(J)+1))


#Define objective
obj1=gp.quicksum(A[i-1]*(1-gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1,len(J)+1))) for i in range(1,len(I)+1))# calcuamte the amount of covered people in the study area
m.setObjective(obj1, GRB.MAXIMIZE)

#Solve the model 
m.optimize()

for v in m.getVars():
       if v.X > 0.01:
        print("%s %g" % (v.Varname, v.X))
print("obj1:", obj1.getValue())
print("finish")

#Based on the opened EMS stations =[1,3] from the below solution, we can find the value of objective 2 (Z2_0)=6*10+2*10=80. And Wr=0.384615
#The result is also same to that of MCLP-original 

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 65 rows, 60 columns and 151 nonzeros
Model fingerprint: 0x4f967988
Variable types: 0 continuous, 60 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 10.0000000
Presolve removed 65 rows and 60 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 16 10 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.600000000000e+01, best bound 1.600000000000e+01, gap 0.0000%
Xj[2] 1
Xj[3] 1
Yij[1,2] 1
Yij[3,2] 1
Yij[4,2] 1
Yij[6,2] 1
Yij[7,2] 1
Yij[8,2] 1
obj1: 16.0
finish


In [13]:
#Model 2. Minimize the total weighted distace for uncovered rural population 
#we aim to locate one EMS station in this scenario (that is q=1)

import gurobipy as gp
from gurobipy import GRB
import numpy as np
m=gp.Model("Urban_rural inequality model_ p median ")

# define decision variables 
X = m.addVars(range(1,len(J)+1),vtype=GRB.BINARY,name="Xj") # covergae decision
Y = m.addVars(range(1,len(I)+1),range(1,len(J)+1),vtype=GRB.INTEGER,lb=0,name="Yij") #Allocation variables

# Define constraints:
#SumXj +Sum(uc)Yij >=1
m.addConstrs(gp.quicksum(X[j]*C[i-1][j-1] for j in range(1,len(J)+1))+
             gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1, len(J)+1))>=1 for i in range(1, len(I)+1))

#Sum Xj=q
m.addConstr(X.sum()==q)
m.addConstr(gp.quicksum(X[j] for j in range(2,3))>=p)

# X[j]-Y[i,j]>=0 for i in I, j in UC
m.addConstrs(X[j]*UC[i-1][j-1]-Y[i,j]*UC[i-1][j-1]>=0 for i in range(1,len(I)+1) for j in range(1,len(J)+1))



#Define the objective
obj2 = gp.quicksum(gp.quicksum(A[i-1]*R[i-1]*D[i-1][j-1]*UC[i-1][j-1]*Y[i,j]for j in range(1, len(J)+1))
              for i in range(1,len(I)+1))#compute the amount of weighted total distacne for uncovered rural population 



m.modelSense = GRB.MINIMIZE
m.setObjective(obj2, GRB.MINIMIZE)

#Solve the model 
m.optimize()


for v in m.getVars():
       if v.X > 0.0001:
        print("%s %g" % (v.Varname, v.X))
print("obj2",obj2.getValue()) 
print("finish")

#Based on the opened EMS stations (3,4), we can find the value of obj 1 (Z1) is ob1=9



Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 65 rows, 60 columns and 151 nonzeros
Model fingerprint: 0xc72d3fd2
Variable types: 0 continuous, 60 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 158.0000000
Presolve removed 65 rows and 60 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 70 158 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.000000000000e+01, best bound 7.000000000000e+01, gap 0.0000%
Xj[2] 1
Xj[4] 1
Yij[1,2] 1
Yij[2,2] 1
Yij[4,2] 1
Yij[5,2] 1
Yij[6,2] 1
Yij[7,2] 1
Yij[8,2] 1
Yij[9,2] 1
obj2 70.0
finish


In [14]:
#Model 3. Minimize the total weight distace for uncovered rural population, while the amuont of covered people should be not less than 10
# we aim to locate one EMS station in thise scenario (that is q=1)

import gurobipy as gp
from gurobipy import GRB
import numpy as np
m=gp.Model("Model 3")

X = m.addVars(range(1,len(J)+1),vtype=GRB.BINARY,name="Xj") # covergae decision
Y = m.addVars(range(1,len(I)+1),range(1,len(J)+1),vtype=GRB.INTEGER,lb=0,name="Yij") #Allocation variables


# Constraints:
#SumXj +Sum(uc)Yij >=1
m.addConstrs(gp.quicksum(X[j]*C[i-1][j-1] for j in range(1,len(J)+1))+
             gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1, len(J)+1))>=1 for i in range(1, len(I)+1))

#Sum Xj=q
m.addConstr(X.sum()==q)
m.addConstr(gp.quicksum(X[j] for j in range(2,3))>=p)


# X[j]-Y[i,j]>=0 for i in I, j in UC
m.addConstrs(X[j]*UC[i-1][j-1]-Y[i,j]*UC[i-1][j-1]>=0 for i in range(1,len(I)+1) for j in range(1,len(J)+1))

#Define objectives

obj2 = gp.quicksum(gp.quicksum(A[i-1]*R[i-1]*D[i-1][j-1]*UC[i-1][j-1]*Y[i,j]for j in range(1, len(J)+1))
              for i in range(1,len(I)+1))

obj1=gp.quicksum(A[i-1]*(1-gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1,len(J)+1))) for i in range(1,len(I)+1))

# e-constraints method
# Due to the value of objective 1 is 25 in Model 2. Thus, we define that the objective 1 should be higher than 25 in this model 
m.addConstr(obj1>=9+1)
#Soleve the model 
m.modelSense = GRB.MINIMIZE
m.setObjective(obj2, GRB.MINIMIZE)
m.optimize()

for v in m.getVars():
       if v.X > 0.001:
        print("%s %g" % (v.Varname, v.X))
print("obj2",obj2.getValue()) 
print("finish")

#Based on the opened EMS stations =[2,3] from the below solution, we can find the value of objective 2 (Z2_0)=6*10+2*10=80；obj1=16

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 66 rows, 60 columns and 196 nonzeros
Model fingerprint: 0x9696184e
Variable types: 0 continuous, 60 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 158.0000000
Presolve removed 66 rows and 60 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 80 158 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.000000000000e+01, best bound 8.000000000000e+01, gap 0.0000%
Xj[2] 1
Xj[3] 1
Yij[1,2] 1
Yij[3,3] 1
Yij[4,2] 1
Yij[6,2] 1
Yij[7,2] 1
Yij[8,2] 1
obj2 80.0
finish


In [15]:
#Model 4.  maximize the total amount of covered demand, while Wr should be higher than the value in Model 1
#we aim to build two EMS stations (that is q=2) 

import gurobipy as gp 
from gurobipy import GRB
import numpy as np
m=gp.Model("Model 4")

#Define decsion variables
X = m.addVars(range(1,len(J)+1),vtype=GRB.BINARY,name="Xj") # covergae decision
Y = m.addVars(range(1,len(I)+1),range(1,len(J)+1),vtype=GRB.INTEGER,lb=0,name="Yij") #All location variables
Wr= m.addVar(vtype= GRB.CONTINUOUS,name="Wr")#the ratio of rural covered population
Wu= m.addVar(vtype=GRB.CONTINUOUS,name="Wu") #the ratio of urban covered popualtion

# Constraints:
#SumXj +Sum(uc)Yij >=1
m.addConstrs(gp.quicksum(X[j]*C[i-1][j-1] for j in range(1,len(J)+1))+
             gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1, len(J)+1))>=1 for i in range(1, len(I)+1))

#Sum Xj=q
q=2   #total number of EMS stations to be built 
m.addConstr(X.sum()==q)
m.addConstr(gp.quicksum(X[j] for j in range(2,3))>=p)

# X[j]-Y[i,j]>=0 for i in I, j in UC
m.addConstrs(X[j]*UC[i-1][j-1]-Y[i,j]*UC[i-1][j-1]>=0 for i in range(1,len(I)+1) for j in range(1,len(J)+1))


#Wu
m.addConstr(gp.quicksum(A[i-1]*(1-R[i-1])*(1-gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1, len(J)+1)))
                        for i in range(1,len(I)+1))/ M -Wu==0)

#Wr
m.addConstr(gp.quicksum(A[i-1]*R[i-1]*(1-gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1, len(J)+1)))
                        for i in range(1,len(I)+1))/ N -Wr==0)

#Define objectives 
obj1=gp.quicksum(A[i-1]*(1-gp.quicksum(Y[i,j]*UC[i-1][j-1] for j in range(1,len(J)+1))) for i in range(1,len(I)+1))

#e-constraints method
#In model 1, Wr=0.384615. Thus, we should define that Wr should be higher than this value in Model 4. 
m.addConstr(Wr>=0.384615+0.001)
#Solve the model 
m.setObjective(obj1, GRB.MAXIMIZE)
m.optimize()
for v in m.getVars():
       if v.X > 0.01:
        print("%s %g" % (v.Varname, v.X)) 
print("obj1:", obj1.getValue())
print("finish")


Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 68 rows, 62 columns and 199 nonzeros
Model fingerprint: 0xf8faa8e5
Variable types: 2 continuous, 60 integer (6 binary)
Coefficient statistics:
  Matrix range     [3e-02, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e-01, 2e+00]
Presolve removed 68 rows and 62 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 9 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.000000000000e+00, best bound 9.000000000000e+00, gap 0.0000%
Xj[2] 1
Xj[4] 1
Yij[1,2] 1
Yij[2,2] 1
Yij[4,2] 1
Yij[5,2] 1
Yij[6,2] 1
Yij[8,2] 1
Yij[9,2] 1
Wr 0.461538
Wu 0.1
obj1: 9.0
finish
